# Experimentation

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import networkx as nx
from parse import *
from utils  import *
from pulp import *
from math import comb
import gurobipy as gp
from gurobipy import GRB

In [2]:
sample_input = pd.read_csv("samples/10.in", header = None)
sample_output = np.array(pd.read_csv("samples/10.out", header = None))

In [3]:
graph, budget = read_input_file("samples/10.in")

In [4]:
pos = nx.kamada_kawai_layout(graph)  # positions for all nodes
# nx.draw(graph, pos, with_labels=True)

In [5]:
# prob = LpProblem("proj",LpMaximize)

In [6]:
# # edges 
# # Add diagonal variables such that (1,1) have 0 happiness and sadness
# edges = np.array(["e01","e02","e03","e12","e13","e23"])
# weights = [9.2, 5.4, 2.123, 75.4,18,87]*3
# sad = [3,40.8,98,8,57.904,9.4]*3
# d = {}
# for j in range(3):
#     for i in edges:
#         d["{0}_{1}".format(i,j)] = LpVariable("{0}_{1}".format(i,j), lowBound = 0, upBound=1, cat="Integer")
# edges = list(d.values())
# # e11 = LpVariable("{0}_{1}".format(i,j), lowBound = 0, upBound=1, cat="Integer")
# # e22 = LpVariable("{0}_{1}".format(i,j), lowBound = 0, upBound=1, cat="Integer")
# # e33 = LpVariable("{0}_{1}".format(i,j), lowBound = 0, upBound=1, cat="Integer")

In [7]:
# # vertices 
# vertices = ["v0", "v1", "v2", "v3"]
# d = {}
# for j in range(3):
#     for i in vertices:
#         d["{0}_{1}".format(i,j)] = LpVariable("{0}_{1}".format(i,j), lowBound = 0, cat="Integer")
# vertices = list(d.values())
# vertices

In [8]:
# prob += sum(edges[i] * weights[i] for i in range(len(weights)))

In [9]:
# prob += sum(edges[i] for i in [0,6,12]) == 1
# prob += sum(edges[i] for i in [1,7,13]) == 1
# prob += sum(edges[i] for i in [2,8,14]) == 1
# prob += sum(edges[i] for i in [3,9,15]) == 1
# prob += sum(edges[i] for i in [4,10,16]) == 1
# prob += sum(edges[i] for i in [5,11,17]) == 1

In [10]:
# prob += sum(edges[i] * sad[i] for i in range(6)) <= 42.314/3
# prob += sum(edges[i] * sad[i] for i in range(6,12)) <= 42.314/3
# prob += sum(edges[i] * sad[i] for i in range(12,18)) <= 42.314/3

# prob += sum(vertices[i] for i in [0,4,8]) == 1 
# prob += sum(vertices[i] for i in [1,5,9]) == 1
# prob += sum(vertices[i] for i in [2,6,10]) == 1
# prob += sum(vertices[i] for i in [3,7,11]) == 1


# # Linearization of quadratic variables (eijk =vik * vjk)

In [11]:
# prob

In [12]:
# prob.solve()
# for v in prob.variables():
#     print(v.name, "=", v.varValue)

# Scale  

Assumptions 
- Rooms with individuals are usually not optimal and hence are not considered in the model
- We have a good heuristic for choosing the number of rooms

<!-- - Either $e_{i,i} = 1$ or $e_{i,j} = 1, \ \ \ \exists j$ -->
- Pick R arbitrarily 
    - Work on heuristics later
<!-- - Add $e_{i,i} \forall \ \ i$ -->
- Create k * (n choose 2) variables representing each pair and its class i.e,  e_i,j,k  for $k \in \{2, n-1\}$
- Objective $max\{\sum e_{i,j,k}*h_{i,j}\}$ 

- Constraints
    - $\sum_{k = 1}^{K} e_{i,j,k} = 1  \ \ \ \ \forall (i,j)$
    - If $(i,j) = 1 \And \ (i,h) = 1 \implies (j,h) = 1$
         - In LP notation: 
    - $\sum_{(i,j)} s_{i,j}e_{i,j,k} \leq \frac{S_{max}}{K} \ \ \ \forall k \in K$ 

- Parse the inputs into H and S arrays representing happiness and sadness 
- Get K number of rooms (improve later)
- Define edges and vertices as LpVariables 
    - Make them into two arrays, one sorted based on ids and another based on rooms (4 arrays in total)
- Define constraints 

In [13]:
n = int(list(sample_input[0])[0])
s_max = int(list(sample_input[0])[1])
K = 3

In [14]:
# Parsing input data 
df = sample_input
df = pd.DataFrame([x.split() for x in df[0].tolist() ])
df = df.loc[2:,:]
df["i"] = df.loc[:,0]
df["j"] = df.loc[:,1]
df["sadness"] = df.loc[:,3]
df["happiness"] = df.loc[:,2]
df = df[["i", "j", "sadness", "happiness"]]

In [15]:
i_id = [int(i) for i in list(df["i"])]
j_id = [int(i) for i in list(df["j"])]
sadness = [float(i) for i in list(df["sadness"])]*K
happiness = [float(i) for i in list(df["happiness"])]*K

sadness_1 = [float(i) for i in list(df["sadness"])]
happiness_1 = [float(i) for i in list(df["happiness"])]
len(sadness) == len(happiness)

True

In [16]:
prob = LpProblem("solver",LpMaximize)

In [17]:
# edges 
d_e = {}
for k in range(K):
    for i in range(comb(n,2)):
        d_e["e_{0}_{1}_{2}".format(i_id[i],j_id[i],k)] = LpVariable("e_{0}_{1}_{2}".format(i_id[i],j_id[i],k), cat="Binary")
edges_1 = list(d_e.values())


# Create another edge array with different sort
edges_2 = [d_e.get(i) for i in sorted(d_e.keys(), key=lambda x: (x[2], x[4]))]

# Create another edge array for constraint 4 
edges_3 = [d_e.get(i) for i in sorted(d_e.keys(), key=lambda x: (x[2], x[6]))]

# Check
len(edges_1) == comb(n,2)*K

True

In [18]:
# vertices 
d_v = {}
for j in range(K):
    for i in range(n):
        d_v["v_{0}_{1}".format(i,j)] = LpVariable("v_{0}_{1}".format(i,j), cat="Binary")
vertices_1 = list(d_v.values())

# Create another edge array with different sort
vertices_2 = [d_v.get(i) for i in sorted(d_v.keys(), key=lambda x: x[2])]

# Check
len(vertices_1) == n*K

True

In [19]:
# Linearization of quadratic variables 1 
v_list = np.array(np.array_split(vertices_1, 3)).tolist()
v_big = []
for i in range(K):
    j = 0
    while j != n:
        for _ in range(j,9):
            v_big.append(v_list[i][j])
        j += 1
        
print(len(edges_1) == len(v_big))

True


In [20]:
# Linearization of quadratic variables 2
v_list_2 = [i[1:] for i in v_list]
v_big_2 = []
for h in range(K):
    j = 0
    while j != n:
        for i in range(j,n-1):
            v_big_2.append(v_list_2[h][i])
        j += 1
        i = j
print(len(edges_1) == len(v_big_2))

True


## Solver 

In [21]:
# Objective function 
prob += sum(edges_1[i] * happiness[i] for i in range(len(happiness)))

In [22]:
# # Not sure if we need this constraint now 
# # Constraint 1 
# constraint_2 = [sum(edges_1[i:i + K]) for i in range(0, len(edges_1), K)]
# for i in range(len(constraint_3)):
#     prob += constraint_2[i] == 1

In [23]:
# Constraint 2
# Each rooms has sadness levels <= s_max / K 
aa = []
h = 0
for i in range(K):
    for j in range(comb(n,2)):
        aa.append(edges_1[h] * sadness_1[j])
        h += 1

constraint_2 = [sum(aa[i:i+ comb(n,2)]) for i in range(0, len(aa), comb(n,2))]
for i in range(len(constraint_2)):
    prob += constraint_2[i] <= s_max / K

In [24]:
# Constraint 3
# Each person belongs to only one room
constraint_3 = [sum(vertices_2[i:i + K]) for i in range(0, len(vertices_2), K)]
for i in range(len(constraint_3)):
    prob += constraint_3[i] == 1

In [25]:
# Linearization of quadratic variables
# Constraint 4.1

for i in range(len(edges_1)):
    prob += edges_1[i] <= v_big[i]
    
# Constraint 4.2
for i in range(len(edges_1)):
    prob += edges_1[i] <= v_big_2[i]


# Constraint 4.3
for i in range(len(edges_1)):
    prob += edges_1[i] >= v_big[i] + v_big_2[i] - 1 
    
# # Constraint 4'
# for i in range(len(edges_1)):
#     prob += edges_1[i] == v_big[i] * v_big_2[i]

In [26]:
path = "/Users/smeetpatel/Library/gurobi.lic"
solver = GUROBI_CMD(path)

In [27]:
prob.solve(solver)
for v in prob.variables():
    print(v.name, "=", v.varValue)

e_0_1_0 = 0.0
e_0_1_1 = 0.0
e_0_1_2 = 0.0
e_0_2_0 = 0.0
e_0_2_1 = 1.0
e_0_2_2 = 0.0
e_0_3_0 = 0.0
e_0_3_1 = 0.0
e_0_3_2 = 0.0
e_0_4_0 = 0.0
e_0_4_1 = 0.0
e_0_4_2 = 0.0
e_0_5_0 = 0.0
e_0_5_1 = 1.0
e_0_5_2 = 0.0
e_0_6_0 = 0.0
e_0_6_1 = 0.0
e_0_6_2 = 0.0
e_0_7_0 = 0.0
e_0_7_1 = 0.0
e_0_7_2 = 0.0
e_0_8_0 = 0.0
e_0_8_1 = 0.0
e_0_8_2 = 0.0
e_0_9_0 = 0.0
e_0_9_1 = 0.0
e_0_9_2 = 0.0
e_1_2_0 = 0.0
e_1_2_1 = 0.0
e_1_2_2 = 0.0
e_1_3_0 = 0.0
e_1_3_1 = 0.0
e_1_3_2 = 0.0
e_1_4_0 = 0.0
e_1_4_1 = 0.0
e_1_4_2 = 0.0
e_1_5_0 = 0.0
e_1_5_1 = 0.0
e_1_5_2 = 0.0
e_1_6_0 = 0.0
e_1_6_1 = 0.0
e_1_6_2 = 0.0
e_1_7_0 = 0.0
e_1_7_1 = 0.0
e_1_7_2 = 1.0
e_1_8_0 = 0.0
e_1_8_1 = 0.0
e_1_8_2 = 0.0
e_1_9_0 = 0.0
e_1_9_1 = 0.0
e_1_9_2 = 1.0
e_2_3_0 = 0.0
e_2_3_1 = 0.0
e_2_3_2 = 0.0
e_2_4_0 = 0.0
e_2_4_1 = 0.0
e_2_4_2 = 0.0
e_2_5_0 = 0.0
e_2_5_1 = 1.0
e_2_5_2 = 0.0
e_2_6_0 = 0.0
e_2_6_1 = 0.0
e_2_6_2 = 0.0
e_2_7_0 = 0.0
e_2_7_1 = 0.0
e_2_7_2 = 0.0
e_2_8_0 = 0.0
e_2_8_1 = 0.0
e_2_8_2 = 0.0
e_2_9_0 = 0.0
e_2_9_1 = 0.0
e_2_9_

# Gurobi Infra 

In [28]:
sample_input = pd.read_csv("samples/10.in", header = None)
n = int(list(sample_input[0])[0])
s_max = int(list(sample_input[0])[1])
K = 3

In [29]:
# Parsing input data 
df = sample_input
df = pd.DataFrame([x.split() for x in df[0].tolist() ])
df = df.loc[2:,:]
df["i"] = df.loc[:,0]
df["j"] = df.loc[:,1]
df["sadness"] = df.loc[:,3]
df["happiness"] = df.loc[:,2]
df = df[["i", "j", "sadness", "happiness"]]

In [30]:
i_id = [int(i) for i in list(df["i"])]
j_id = [int(i) for i in list(df["j"])]
sadness = [float(i) for i in list(df["sadness"])]*K
happiness = [float(i) for i in list(df["happiness"])]*K

sadness_1 = [float(i) for i in list(df["sadness"])]
happiness_1 = [float(i) for i in list(df["happiness"])]
len(sadness) == len(happiness)

True

In [31]:
m = gp.Model("proj")

Using license file /Users/smeetpatel/gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-22


In [32]:
d_e = {}
for k in range(K):
    for i in range(comb(n,2)):
        d_e["e_{0}_{1}_{2}".format(i_id[i],j_id[i],k)] = m.addVar(name= "e_{0}_{1}_{2}".format(i_id[i],j_id[i],k), vtype = GRB.BINARY)
edges_1 = list(d_e.values())

In [33]:
# Create another edge array with different sort
edges_2 = [d_e.get(i) for i in sorted(d_e.keys(), key=lambda x: (x[2], x[4]))]

# Create another edge array for constraint 4 
edges_3 = [d_e.get(i) for i in sorted(d_e.keys(), key=lambda x: (x[2], x[6]))]

# Check
len(edges_1) == comb(n,2)*K

True

In [34]:
# vertices 
d_v = {}
for j in range(K):
    for i in range(n):
        d_v["v_{0}_{1}".format(i,j)] = m.addVar(name = "v_{0}_{1}".format(i,j),  vtype = GRB.BINARY)
vertices_1 = list(d_v.values())

# Create another edge array with different sort
vertices_2 = [d_v.get(i) for i in sorted(d_v.keys(), key=lambda x: x[2])]

# Check
len(vertices_1) == n*K

True

In [35]:
# Linearization of quadratic variables 1 
v_list = np.array(np.array_split(vertices_1, K)).tolist()
v_big = []
for i in range(K):
    j = 0
    while j != n:
        for _ in range(j,n-1):
            v_big.append(v_list[i][j])
        j += 1
        
print(len(edges_1) == len(v_big))

True


In [36]:
# Linearization of quadratic variables 2
v_list_2 = [i[1:] for i in v_list]
v_big_2 = []
for h in range(K):
    j = 0
    while j != n:
        for i in range(j,n-1):
            v_big_2.append(v_list_2[h][i])
        j += 1
        i = j
print(len(edges_1) == len(v_big_2))

True


In [37]:
m.setObjective(sum(edges_1[i] * happiness[i] for i in range(len(happiness))), GRB.MAXIMIZE)

In [38]:
# # Not sure if we need this constraint now 
# # Constraint 1 
# constraint_2 = [sum(edges_1[i:i + K]) for i in range(0, len(edges_1), K)]
# for i in range(len(constraint_3)):
#     m.addConstr(constraint_2[i] == 1)

In [39]:
aa = []
h = 0
for i in range(K):
    for j in range(comb(n,2)):
        aa.append(edges_1[h] * sadness_1[j])
        h += 1

constraint_2 = [sum(aa[i:i+ comb(n,2)]) for i in range(0, len(aa), comb(n,2))]
for i in range(len(constraint_2)):
    m.addConstr(constraint_2[i] <=  s_max / K)

In [40]:
constraint_3 = [sum(vertices_2[i:i + K]) for i in range(0, len(vertices_2), K)]
for i in range(len(constraint_3)):
    m.addConstr(constraint_3[i] == 1)

In [41]:
for i in range(len(edges_1)):
     m.addConstr(edges_1[i] <= v_big[i])
    
# Constraint 4.2
for i in range(len(edges_1)):
     m.addConstr(edges_1[i] <= v_big_2[i])


# Constraint 4.3
for i in range(len(edges_1)):
    m.addConstr(edges_1[i] >= v_big[i] + v_big_2[i] - 1)

# # Constraint 4'
# for i in range(len(edges_1)):
#     m.addConstr(edges_1[i] == v_big[i] * v_big_2[i])

In [42]:
m.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 418 rows, 165 columns and 1110 nonzeros
Model fingerprint: 0x40f90fe8
Variable types: 0 continuous, 165 integer (165 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+01]
  Objective range  [2e+00, 1e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve time: 0.01s
Presolved: 418 rows, 165 columns, 1110 nonzeros
Variable types: 0 continuous, 165 integer (165 binary)
Found heuristic solution: objective 81.0000000

Root relaxation: objective 1.068825e+02, 151 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  106.88248    0   66   81.00000  106.88248  32.0%     -    0s
     0     0   99.69314    0   73   81.00000   99.69314  23.1%     -    0s
     0     0   95.

In [43]:
for v in m.getVars():
    print((v.varName , v.x))

('e_0_1_0', 0.0)
('e_0_2_0', 1.0)
('e_0_3_0', 0.0)
('e_0_4_0', 0.0)
('e_0_5_0', 1.0)
('e_0_6_0', 0.0)
('e_0_7_0', 0.0)
('e_0_8_0', 0.0)
('e_0_9_0', 0.0)
('e_1_2_0', 0.0)
('e_1_3_0', 0.0)
('e_1_4_0', 0.0)
('e_1_5_0', 0.0)
('e_1_6_0', 0.0)
('e_1_7_0', 0.0)
('e_1_8_0', 0.0)
('e_1_9_0', 0.0)
('e_2_3_0', 0.0)
('e_2_4_0', 0.0)
('e_2_5_0', 1.0)
('e_2_6_0', 0.0)
('e_2_7_0', 0.0)
('e_2_8_0', 0.0)
('e_2_9_0', 0.0)
('e_3_4_0', 0.0)
('e_3_5_0', 0.0)
('e_3_6_0', 0.0)
('e_3_7_0', 0.0)
('e_3_8_0', 0.0)
('e_3_9_0', 0.0)
('e_4_5_0', 0.0)
('e_4_6_0', 0.0)
('e_4_7_0', 0.0)
('e_4_8_0', 0.0)
('e_4_9_0', 0.0)
('e_5_6_0', 0.0)
('e_5_7_0', 0.0)
('e_5_8_0', 0.0)
('e_5_9_0', 0.0)
('e_6_7_0', 0.0)
('e_6_8_0', 0.0)
('e_6_9_0', 0.0)
('e_7_8_0', 0.0)
('e_7_9_0', 0.0)
('e_8_9_0', 0.0)
('e_0_1_1', 0.0)
('e_0_2_1', 0.0)
('e_0_3_1', 0.0)
('e_0_4_1', 0.0)
('e_0_5_1', 0.0)
('e_0_6_1', 0.0)
('e_0_7_1', 0.0)
('e_0_8_1', 0.0)
('e_0_9_1', 0.0)
('e_1_2_1', 0.0)
('e_1_3_1', 0.0)
('e_1_4_1', 0.0)
('e_1_5_1', 0.0)
('e_1_6_1', 0.